In [1]:
!pip install backtrader

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as st

import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

import math
import datetime
import pytz

# from ipynb.fs.full.Task1 import get_files
# from ipynb.fs.full.Task2 import find_suitable_pairs
from BackTrader import *
# from BackTrader import PandasData, PairTradingStrategy
# from BackTrader import CommInfoFloat, DataAnalyzer, CashValueAnalyzer, OrderAnalyzer

In [3]:
# import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers
from enum import Enum
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib

import math
import datetime

#This is the data feed to be fed into the backtesting lib
class PandasData(btfeeds.PandasData):
    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Adj Close'),
        ('openinterest', None),
    )
    
class Log(btind.Indicator):
    """Calculates log."""
    lines = ('log',)
    
    def next(self):
        self.l.log[0] = math.log(self.data[0])

class OLSSlopeIntercept(btind.PeriodN):
    """Calculates a linear regression using OLS."""
    _mindatas = 2  # ensure at least 2 data feeds are passed

    lines = ('slope', 'intercept',)
    params = (
        ('period', 10),
    )

    def next(self):
        p0 = pd.Series(self.data0.get(size=self.p.period))
        p1 = pd.Series(self.data1.get(size=self.p.period))
        p1 = sm.add_constant(p1)
        intercept, slope = sm.OLS(p0, p1).fit().params

        self.lines.slope[0] = slope
        self.lines.intercept[0] = intercept

class OLSSpread(btind.PeriodN):
    """Calculates the z-score of the OLS spread."""
    _mindatas = 2  # ensure at least 2 data feeds are passed
    lines = ('slope', 'spread', 'spread_mean', 'spread_std', 'zscore',)
    params = (('period', 10),)

    def __init__(self):
        data0_log = Log(self.data0)
        data1_log = Log(self.data1)
        slint = OLSSlopeIntercept(data0_log, data1_log, period=self.p.period)

        spread = data0_log - (slint.slope * data1_log + slint.intercept)
        self.l.spread = spread
        self.l.slope = slint.slope

        self.l.spread_mean = bt.ind.SMA(spread, period=self.p.period)
        self.l.spread_std = bt.ind.StdDev(spread, period=self.p.period)
        self.l.zscore = (spread - self.l.spread_mean) / self.l.spread_std
        
class Status(Enum):
    LONG = 1
    SHORT = 2
    NONE = 3
    
class PairTradingStrategy(bt.Strategy):
    """Basic pair trading strategy."""
    
    # These are just default values, pass in these values while initialising strategy
    params = dict(
        period=100,
        order_pct1=0.1,
        order_pct2=0.1,
        printout=True,
        upper=2,
        lower=2,
        symbol1="SYMBOL1",
        symbol2="SYMBOL2"
    )

    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE {}, size = {:.2f}, price = {:.2f}'.format(
                    order.data._name, order.executed.size, order.executed.price)
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE {}, size = {:.2f}, price = {:.2f}'.format(
                    order.data._name, order.executed.size, order.executed.price)
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None
        self.order_pct1 = self.p.order_pct1
        self.order_pct2 = self.p.order_pct2
        self.upper = self.p.upper
        self.lower = self.p.lower
        
        self.transform = OLSSpread(self.data0, self.data1, period=self.p.period)

        self.spread = self.transform.spread
        self.zscore = self.transform.zscore
        self.slope = self.transform.slope
        
        self.status = Status.NONE
        self.symbol1 = self.p.symbol1
        self.symbol2 = self.p.symbol2

    def next(self):
        if self.orderid:
            return  # if an order is active, no new orders are allowed
        
        if self.zscore[0] > self.upper and self.status != Status.SHORT:
            # Short sell stock 1
            self.order_target_percent(data=self.data0, target=-self.order_pct1)
            # Buy stock 2
            self.order_target_percent(data=self.data1, target=self.order_pct2)
            
            self.status = Status.SHORT

        elif self.zscore[0] < self.lower and self.status != Status.LONG:
            # Short sell stock 2
            self.order_target_percent(data=self.data1, target=-self.order_pct2)
            # Buy stock 1
            self.order_target_percent(data=self.data0, target=self.order_pct1)
                     
            self.status = Status.LONG
        
#         # Sample strategy does not close position when zscore reaches zero again, 
#         # only reverses position when it swings the other way
#         # uncomment below if desired behaviour is to close at z-score = 0
#         elif self.zscore[0] <= 0 and self.status == Status.SHORT:
#             # Close position
#             self.order_target_percent(data=self.data1, target=0)
#             self.order_target_percent(data=self.data0, target=0)
#             self.status = Status.NONE
        
#         elif self.zscore[0] >= 0 and self.status == Status.LONG:
#             # Close position
#             self.order_target_percent(data=self.data1, target=0)
#             self.order_target_percent(data=self.data0, target=0)
#             self.status = Status.NONE

    def stop(self):
        if self.p.printout:
            print('==================================================')
            print('Starting Value - %.2f' % self.broker.startingcash)
            print('Ending   Value - %.2f' % self.broker.getvalue())
            print('==================================================')


# This this can be added to Cerebro allow fractional shares (for this like crypto)
# By default backtrader doesnt allow that.
class CommInfoFloat(bt.CommInfoBase):
    """Commission schema that keeps size as float."""
    params = (
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_PERC),
        ('percabs', True),
      )
    
    def getsize(self, price, cash):
        if not self._stocklike:
            return self.p.leverage * (cash / self.get_margin(price))

        return self.p.leverage * (cash / price)


class DataAnalyzer(bt.analyzers.Analyzer):
    """Analyzer to extract OHLCV."""
    def create_analysis(self):
        self.rets0 = {}
        self.rets1 = {}

    def next(self):
        self.rets0[self.strategy.datetime.datetime()] = [
            self.data0.open[0],
            self.data0.high[0],
            self.data0.low[0],
            self.data0.close[0],
            self.data0.volume[0]
        ]
        self.rets1[self.strategy.datetime.datetime()] = [
            self.data1.open[0],
            self.data1.high[0],
            self.data1.low[0],
            self.data1.close[0],
            self.data1.volume[0]
        ]

    def get_analysis(self):
        return self.rets0, self.rets1

class CashValueAnalyzer(bt.analyzers.Analyzer):
    """Analyzer to extract cash and value."""
    def create_analysis(self):
        self.rets = {}

    def notify_cashvalue(self, cash, value):
        self.rets[self.strategy.datetime.datetime()] = self.strategy.broker.getvalue()

    def get_analysis(self):
        return self.rets
    
class OrderAnalyzer(bt.analyzers.Analyzer):
    """Analyzer to extract order price, size, value, and paid commission."""
    def create_analysis(self):
        self.rets0 = {}
        self.rets1 = {}

    def notify_order(self, order):
        if order.status == order.Completed:
            if order.data._name == self.strategy.symbol1:
                rets = self.rets0
            else:
                rets = self.rets1
            rets[self.strategy.datetime.datetime()] = (
                order.executed.price,
                order.executed.size,
                -order.executed.size * order.executed.price,
                order.executed.comm
            )

    def get_analysis(self):
        return self.rets0, self.rets1

In [4]:
def calculate_volatility(values):
    values = pd.Series(values)
    log_return = np.log(values/values.shift())
    volatility = log_return.std()*252**.5
    return volatility*100

In [5]:
start_date = '2015-01-01'
end_date = '2023-01-01'

In [6]:
import yfinance as yf
CSI300 = yf.download("000300.SS", start=start_date, end=end_date)
CSI300 = CSI300.drop(columns=['Volume'])
CSI300

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close
Date,,,,,
2021-03-11,5024.560059,5138.410156,5020.580078,5128.220215,5128.220215
2021-03-12,5153.669922,5153.669922,5086.819824,5146.379883,5146.379883
2021-03-15,5116.120117,5120.879883,4992.399902,5035.540039,5035.540039
2021-03-16,5054.410156,5084.310059,5009.950195,5079.359863,5079.359863
2021-03-17,5062.770020,5123.549805,5020.129883,5100.859863,5100.859863
...,...,...,...,...,...
2022-12-26,3830.570068,3856.810059,3827.520020,3843.489990,3843.489990
2022-12-27,3860.909912,3891.850098,3856.580078,3887.850098,3887.850098
2022-12-28,3874.909912,3884.719971,3854.939941,3871.260010,3871.260010


In [7]:
class BuyAndHold(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        self.order_target_value(target=self.broker.get_cash())

    def stop(self):
        pass
#         # calculate the actual returns
#         self.roi = (self.broker.get_value() / self.val_start) - 1.0
#         print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [8]:
cerebro = bt.Cerebro()
csi300_datafeed = PandasData(dataname=CSI300, name="CSI300")
cerebro.adddata(csi300_datafeed)
cerebro.addstrategy(BuyAndHold)
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.SharpeRatio_A, _name='sharpe')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.broker.setcash(100000)
res = cerebro.run()[0]
print("Sharpe Ratio: ", res.analyzers.sharpe.get_analysis()["sharperatio"])
print("Annualised Return: ", res.analyzers.returns.get_analysis()["rnorm100"])
print("Max Drawdown: ", res.analyzers.drawdown.get_analysis()["max"]["drawdown"])
print("Annualised Volatilty: ", calculate_volatility(CSI300["Close"]))
# print()
# pairs.at[indexes[index], "SharpeRatio"] = res.analyzers.sharpe.get_analysis()["sharperatio"]
#         pairs.at[indexes[index], "Annualised Return(%)"] = res.analyzers.returns.get_analysis()["rnorm100"]
#         pairs.at[indexes[index], "Max Drawdown(%)"] = res.analyzers.drawdown.get_analysis()["max"]["drawdown"]
#         pairs.at[indexes[index], "Annualised Volatility(%)"] = volatility

Sharpe Ratio:  -1.5905762618206447
Annualised Return:  -14.714349933850542
Max Drawdown:  33.62545741806995
Annualised Volatilty:  18.71546363098488


In [9]:
PRICE_DIR = "individual_data"
STOCK_PRICE = {}

In [10]:
# Adjust these params for backtesting
PARAMS = {
    "CASH" : 100000,
    "UPPER" : st.norm.ppf(1 - 0.05 / 2),
    "LOWER" : -st.norm.ppf(1 - 0.05 / 2),
    "ORDER_PCT1" : 0.5, # Essentially order size (how much pct of current portfolio you want to spend for stock)
    "ORDER_PCT2" : 0.5,
    "PERIOD" : 100, # Look back period for doing rolling OLS and z-score calc.
    "COMMPERC": 0.005 # 0.5%
}

def run_backtest(stock1_name, stock2_name, stock1_df, stock2_df, PARAMS):
    stock1_datafeed = PandasData(dataname=stock1_df, name=stock1_name)
    stock2_datafeed = PandasData(dataname=stock2_df, name=stock2_name)

    
    cerebro = bt.Cerebro()

    cerebro.adddata(stock1_datafeed)
    cerebro.adddata(stock2_datafeed)

    cerebro.addstrategy(PairTradingStrategy, 
                        period=PARAMS["PERIOD"], 
                        upper=PARAMS["UPPER"], 
                        lower=PARAMS["LOWER"], 
                        order_pct1=PARAMS["ORDER_PCT1"],
                        order_pct2=PARAMS["ORDER_PCT2"],
                        symbol1=stock1_name,
                        symbol2=stock2_name,
                        printout=False)

    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.SharpeRatio_A, _name='sharpe')
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='ta')
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
#     cerebro.addanalyzer(btanalyzers.LogReturnsRolling, _name='logrollingreturns')
    cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
#     cerebro.addanalyzer(btanalyzers.PositionsValue, _name='positionvalues')
    
    # Below are custom analyzers defined by the sample code, can uncomment if you need more detailed info
    cerebro.addanalyzer(DataAnalyzer)
    cerebro.addanalyzer(CashValueAnalyzer)
    cerebro.addanalyzer(OrderAnalyzer)

    cerebro.broker.setcash(PARAMS["CASH"])
    comminfo = CommInfoFloat(commission=PARAMS["COMMPERC"])
    cerebro.broker.addcommissioninfo(comminfo)

    backtest_res = cerebro.run()[0]
    return backtest_res

In [11]:
filtered_pairs = pd.read_pickle("Data/filtered_pairs_dict.pickle")

In [12]:
filtered_pairs

{'KMeans':                                                    pair        pvalue  \
 0     (Communication Services-Communication Services...  1.024104e-03   
 2     (Communication Services-Communication Services...  1.370479e-05   
 5     (Healthcare-Healthcare - 300142.SZ.csv, Health...  2.835066e-02   
 6     (Healthcare-Healthcare - 300142.SZ.csv, Indust...  1.871553e-02   
 7     (Healthcare-Healthcare - 300142.SZ.csv, Indust...  2.565211e-02   
 ...                                                 ...           ...   
 3310  (Industry-Industry - 002709.SZ.csv, Industry-I...  0.000000e+00   
 3314  (Industry-Industry - 300014.SZ.csv, Industry-I...  0.000000e+00   
 3315  (Industry-Industry - 300014.SZ.csv, Materials-...  0.000000e+00   
 3316  (Industry-Industry - 300014.SZ.csv, Materials-...  0.000000e+00   
 3317  (Industry-Industry - 600884.SH.csv, Materials-...  1.813381e-29   
 
       hurst_exp   half_life  avg_cross_count  
 0      0.045492    1.693115       111.770021  
 2  

In [13]:
start_test = '2021-01-01'
end_test = '2022-12-30'

In [14]:
from datetime import datetime

In [16]:
TESTING_START = datetime(2021, 1, 1)
TESTING_END = datetime(2022, 12, 30)
backtest_results_raw = {}

In [17]:
STOCK_PRICE = {}

In [18]:
def read_stock_price(stock_name):
    stock_name = stock_name.replace("-", "/", 1)
    if stock_name in STOCK_PRICE:
        return STOCK_PRICE[stock_name]
    file_path = PRICE_DIR + "/" + stock_name
    df = pd.read_csv(file_path)
    df = df.drop(0)
    df = df.ffill()
    dates = pd.to_datetime(df['Date'], format="%Y%m%d", utc=True)
    dates = dates.dt.strftime("%Y-%m-%d")
    df['Date'] = dates
    dates = pd.to_datetime(dates)
    
#     print(dates)
#     print(df['Date'])
    df.index = dates
    df = df.loc[df.index >= TESTING_START]
    df = df.loc[df.index <= TESTING_END]
    STOCK_PRICE[stock_name] = df
    
    df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'adj close': 'Adj Close', 'vol': 'Volume'}, inplace=True)
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Returns']]
    return df

In [269]:
for pair_strat, pairs in filtered_pairs.items():
    print("Doing Backtesting for ", pair_strat)
    pairs["SharpeRatio"] = np.nan
    pairs["Annualised Return(%)"] = np.nan
    pairs["Max Drawdown(%)"] = np.nan
    pairs["Annualised Volatility(%)"] = np.nan
    raw_results = {}
    indexes = pairs.index
    for (index, (stock1, stock2)) in enumerate(pairs["pair"]):
        print("Backtesting for ", (stock1, stock2))
        stock1_df = read_stock_price(stock1)
        stock2_df = read_stock_price(stock2)
        if stock1_df.isnull().values.any() or stock2_df.isnull().values.any():
            print(f'{stock1} or {stock2} has null values')
            continue
        try:
            res = run_backtest(stock1, stock2, stock1_df, stock2_df, PARAMS)
        except IndexError:
            print('Index Error')
        portfolio_value = res.analyzers.cashvalueanalyzer.get_analysis()
        volatility = calculate_volatility(portfolio_value.values())
        orders = res.analyzers.orderanalyzer.get_analysis()
        
        # Raw results store the portfolio value at each date, and also the specific orders which were made during backtesting
        raw_results[(stock1, stock2)] = (portfolio_value, orders)        
        print(res.analyzers.sharpe.get_analysis())
        break
        pairs.at[indexes[index], "SharpeRatio"] = res.analyzers.sharpe.get_analysis()["sharperatio"]
        pairs.at[indexes[index], "Annualised Return(%)"] = res.analyzers.returns.get_analysis()["rnorm100"]
        pairs.at[indexes[index], "Max Drawdown(%)"] = res.analyzers.drawdown.get_analysis()["max"]["drawdown"]
        pairs.at[indexes[index], "Annualised Volatility(%)"] = volatility
#     backtest_results_raw[pair_strat] = raw_results


Doing Backtesting for  KMeans
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Communication Services-Communication Services - 002555.SZ.csv')
OrderedDict([('sharperatio', None)])
Doing Backtesting for  Agglo
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Communication Services-Communication Services - 002555.SZ.csv')
OrderedDict([('sharperatio', None)])
Doing Backtesting for  DBSCAN
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Communication Services-Communication Services - 300628.SZ.csv')
OrderedDict([('sharperatio', None)])


In [268]:
raw_results


{('Communication Services-Communication Services - 000063.SZ.csv',
  'Communication Services-Communication Services - 300628.SZ.csv'): ({datetime.datetime(2023, 1, 3, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 4, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 5, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 6, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 9, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 10, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 11, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 12, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 13, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 16, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 17, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 18, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 19, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 20, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 30, 0, 0): 100000.0,
   datetime.datetime(2023, 1, 31, 0, 0): 100000.0,
   datetime.datetime(2023, 2, 1, 0, 0): 100000.0,
   da

In [19]:
for pair_strat, pairs in filtered_pairs.items():
    print("Doing Backtesting for ", pair_strat)
    pairs["SharpeRatio"] = np.nan
    pairs["Annualised Return(%)"] = np.nan
    pairs["Max Drawdown(%)"] = np.nan
    pairs["Annualised Volatility(%)"] = np.nan
    raw_results = {}
    indexes = pairs.index
    for (index, (stock1, stock2)) in enumerate(pairs["pair"]):
        print("Backtesting for ", (stock1, stock2))
        stock1_df = read_stock_price(stock1)
        stock2_df = read_stock_price(stock2)
        if stock1_df.isnull().values.any() or stock2_df.isnull().values.any():
            print(f'{stock1} or {stock2} has null values')
            continue
        try:
            res = run_backtest(stock1, stock2, stock1_df, stock2_df, PARAMS)
        except IndexError:
            print('Index Error')
        portfolio_value = res.analyzers.cashvalueanalyzer.get_analysis()
        volatility = calculate_volatility(portfolio_value.values())
        orders = res.analyzers.orderanalyzer.get_analysis()
        
        # Raw results store the portfolio value at each date, and also the specific orders which were made during backtesting
        raw_results[(stock1, stock2)] = (portfolio_value, orders)        
        
        pairs.at[indexes[index], "SharpeRatio"] = res.analyzers.sharpe.get_analysis()["sharperatio"]
        pairs.at[indexes[index], "Annualised Return(%)"] = res.analyzers.returns.get_analysis()["rnorm100"]
        pairs.at[indexes[index], "Max Drawdown(%)"] = res.analyzers.drawdown.get_analysis()["max"]["drawdown"]
        pairs.at[indexes[index], "Annualised Volatility(%)"] = volatility
    backtest_results_raw[pair_strat] = raw_results

        

Doing Backtesting for  KMeans
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Healthcare-Healthcare - 300142.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 000786.SZ.csv')
Backtesting for  ('Healthcare-Healthcare - 300142.SZ.csv', 'Healthcare-Healthcare - 600763.SH.csv')
Backtesting for  ('Healthcare-Healthcare - 300142.SZ.csv', 'Industry-Industry - 000786.SZ.csv')
Backtesting for  ('Healthcare-Healthcare - 300142.SZ.csv', 'Industry-Industry - 601919.SH.csv')
Backtesting for  ('Healthcare-Healthcare - 300142.SZ.csv', 'Materials-Materials - 600346.SH.csv')
Backtesting for  ('Energy-Energy - 000723.SZ.csv', 'Energy-Energy - 000983.SZ.csv')
Index Error
Backtesting for  ('Energy-Energy - 000723.SZ.csv', 'Energy-Energy - 600188.SH.csv')
Backtesting for  ('Energy-Energy - 000723.SZ.csv', 'Energy-Energy - 601225.SH.csv')
Backtesting for  ('Energy-Energy - 000723.SZ.csv', 'Energy-Energy - 6016

Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 000877.SZ.csv')
Index Error
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 002064.SZ.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 002493.SZ.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 600010.SH.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 600111.SH.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 600219.SH.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 600362.SH.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 600426.SH.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 600547.SH.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 601899.SH.csv')
Backtesting for  ('Energy-Energy - 600188.SH.csv', 'Materials-Materials - 603993.SH.csv'

Index Error
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Industry-Industry - 600089.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Industry-Industry - 600150.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Industry-Industry - 600875.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Industry-Industry - 600893.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Industry-Industry - 601117.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Industry-Industry - 601872.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Industry-Industry - 601989.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Information Technology-Information Technology - 601360.SH.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Materials-Materials - 000301.SZ.csv')
Index Error
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Materials-Materials - 000708.SZ.csv')
Backtesting for  ('Energy-Energy - 601898.SH.csv', 'Materials-Ma

Backtesting for  ('Industry-Industry - 000338.SZ.csv', 'Utilities-Utilities - 600803.SH.csv')
Index Error
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Industry-Industry - 000768.SZ.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Industry-Industry - 000800.SZ.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Industry-Industry - 600031.SH.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Industry-Industry - 600089.SH.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Industry-Industry - 600760.SH.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Industry-Industry - 600893.SH.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Industry-Industry - 601618.SH.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Materials-Materials - 000301.SZ.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Materials-Materials - 000408.SZ.csv')
Backtesting for  ('Industry-Industry - 000425.SZ.csv', 'Materi

Index Error
Backtesting for  ('Industry-Industry - 600039.SH.csv', 'Materials-Materials - 000877.SZ.csv')
Backtesting for  ('Industry-Industry - 600039.SH.csv', 'Materials-Materials - 600010.SH.csv')
Backtesting for  ('Industry-Industry - 600039.SH.csv', 'Materials-Materials - 600111.SH.csv')
Backtesting for  ('Industry-Industry - 600039.SH.csv', 'Materials-Materials - 600219.SH.csv')
Backtesting for  ('Industry-Industry - 600039.SH.csv', 'Materials-Materials - 600426.SH.csv')
Backtesting for  ('Industry-Industry - 600039.SH.csv', 'Materials-Materials - 600547.SH.csv')
Backtesting for  ('Industry-Industry - 600039.SH.csv', 'Materials-Materials - 603993.SH.csv')
Backtesting for  ('Industry-Industry - 600089.SH.csv', 'Industry-Industry - 600150.SH.csv')
Backtesting for  ('Industry-Industry - 600089.SH.csv', 'Industry-Industry - 600893.SH.csv')
Backtesting for  ('Industry-Industry - 600089.SH.csv', 'Industry-Industry - 601117.SH.csv')
Backtesting for  ('Industry-Industry - 600089.SH.csv',

...

Backtesting for  ('Finance-Finance - 600030.SH.csv', 'Finance-Finance - 601336.SH.csv')
Backtesting for  ('Finance-Finance - 600030.SH.csv', 'Finance-Finance - 601688.SH.csv')
Backtesting for  ('Finance-Finance - 600030.SH.csv', 'Finance-Finance - 601788.SH.csv')
Backtesting for  ('Finance-Finance - 600030.SH.csv', 'Real Estate-Real Estate - 000002.SZ.csv')
Backtesting for  ('Finance-Finance - 600837.SH.csv', 'Finance-Finance - 600999.SH.csv')
Backtesting for  ('Finance-Finance - 600837.SH.csv', 'Finance-Finance - 601336.SH.csv')
Backtesting for  ('Finance-Finance - 600837.SH.csv', 'Finance-Finance - 601377.SH.csv')
Index Error
Backtesting for  ('Finance-Finance - 600837.SH.csv', 'Finance-Finance - 601688.SH.csv')
Backtesting for  ('Finance-Finance - 600837.SH.csv', 'Finance-Finance - 601788.SH.csv')
Backtesting for  ('Finance-Finance - 600837.SH.csv', 'Information Technology-Information Technology - 300033.SZ.csv')
Backtesting for  ('Finance-Finance - 600999.SH.csv', 'Finance-Finance 

Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 600406.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 600760.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 600875.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 600893.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 601117.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 601989.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Industry-Industry - 603806.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 002027.SZ.csv', 'Information Technology-Information Technology - 300033.SZ.csv')
Backtesting 

Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Industry-Industry - 601800.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Industry-Industry - 601989.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Information Technology-Information Technology - 000100.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Information Technology-Information Technology - 000733.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Information Technology-Information Technology - 601360.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Information Technology-Information Technology - 603019.SH.csv')
Backtesting for  ('Communication Services-Communication Services - 000063.SZ.csv', 'Materials-Materials - 000301.SZ.csv')
Index Error
Backtesting for  ('Communication S

Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Healthcare-Healthcare - 000661.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Healthcare-Healthcare - 000963.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Healthcare-Healthcare - 002001.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Healthcare-Healthcare - 002007.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Healthcare-Healthcare - 300015.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Healthcare-Healthcare - 300122.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Healthcare-Healthcare - 300142.SZ.csv')
Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Industry-Industry - 000157.SZ.csv')
Backtesting 

Backtesting for  ('Communication Services-Communication Services - 600050.SH.csv', 'Utilities-Utilities - 600803.SH.csv')
Index Error


In [20]:
import pickle
pickle.dump(filtered_pairs, open('results_df.pickle', 'wb'))
pickle.dump(backtest_results_raw, open('results_raw.pickle', 'wb'))

In [1]:
import pickle

In [2]:
test = pickle.load(open('results_df.pickle', 'rb'))

In [3]:
test

{'KMeans':                                                    pair        pvalue  \
 0     (Communication Services-Communication Services...  1.024104e-03   
 2     (Communication Services-Communication Services...  1.370479e-05   
 5     (Healthcare-Healthcare - 300142.SZ.csv, Health...  2.835066e-02   
 6     (Healthcare-Healthcare - 300142.SZ.csv, Indust...  1.871553e-02   
 7     (Healthcare-Healthcare - 300142.SZ.csv, Indust...  2.565211e-02   
 ...                                                 ...           ...   
 3310  (Industry-Industry - 002709.SZ.csv, Industry-I...  0.000000e+00   
 3314  (Industry-Industry - 300014.SZ.csv, Industry-I...  0.000000e+00   
 3315  (Industry-Industry - 300014.SZ.csv, Materials-...  0.000000e+00   
 3316  (Industry-Industry - 300014.SZ.csv, Materials-...  0.000000e+00   
 3317  (Industry-Industry - 600884.SH.csv, Materials-...  1.813381e-29   
 
       hurst_exp   half_life  avg_cross_count  SharpeRatio  \
 0      0.045492    1.693115       111

In [4]:
test['KMeans']

,pair,pvalue,hurst_exp,half_life,avg_cross_count,SharpeRatio,Annualised Return(%),Max Drawdown(%),Annualised Volatility(%)
0,(Communication Services-Communication Services...,1.024104e-03,0.045492,1.693115,111.770021,-0.317954,-0.801610,28.303495,22.794377
2,(Communication Services-Communication Services...,1.370479e-05,0.028210,2.532232,111.252567,-0.022183,0.343329,20.664132,16.446214
5,"(Healthcare-Healthcare - 300142.SZ.csv, Health...",2.835066e-02,0.066402,4.820998,96.936345,0.914259,11.509316,16.114526,16.608799
6,"(Healthcare-Healthcare - 300142.SZ.csv, Indust...",1.871553e-02,0.028967,8.024377,103.663244,0.223169,1.329542,8.803951,9.481991
7,"(Healthcare-Healthcare - 300142.SZ.csv, Indust...",2.565211e-02,0.097647,8.811799,106.595483,-1.441624,-2.379265,12.166765,15.921455
...,...,...,...,...,...,...,...,...,...
3310,"(Industry-Industry - 002709.SZ.csv, Industry-I...",0.000000e+00,0.000400,157.778025,128.845996,-0.928773,-4.339996,31.344279,17.586267
3314,"(Industry-Industry - 300014.SZ.csv, Industry-I...",0.000000e+00,0.005210,11.634979,126.776181,0.682757,10.917467,22.495830,15.769745
3315,"(Industry-Industry - 300014.SZ.csv, Materials-...",0.000000e+00,0.007635,51.272003,124.188912,-0.291994,-0.381570,26.338041,17.427966
3316,"(Industry-Industry - 300014.SZ.csv, Materials-...",0.000000e+00,0.006595,59.069991,119.531828,2.597981,7.084105,24.367752,20.618151


In [5]:
kmeans = test['KMeans']

In [17]:
kmeans[(kmeans['SharpeRatio'] > 1.5) & (kmeans['SharpeRatio'] < 10)].to_csv("kmeans_chinamarket.csv")

In [8]:
dbscan = test['DBSCAN']

In [9]:
len(dbscan)

165

In [18]:
dbscan[dbscan['SharpeRatio'] > 1.5].to_csv("dbscan_chinamarket.csv")

In [11]:
agglo = test['Agglo']

In [19]:
agglo[agglo['SharpeRatio'] > 1.5].to_csv("agglo_chinamarket.csv")

In [ ]:
# def read_stock_price(file_path: str):
#     df = pd.read_csv(file_path, header=[0, 1]).drop(0)
#     # Skipping files with null values for now
#     if df.isnull().values.any():
#         print(f'{file_path} has null values')
#         return None
#     dates = pd.to_datetime(df.iloc[:, 0], utc=True).dt.date
#     dates = pd.to_datetime(dates)
#     dates.name = "Date"
#     df.index = dates
#     return df

In [ ]:
# prices = {}
# files = get_files(path="Data", extension="csv")
# for file_name, file_path in files.items():
#     df = read_stock_price(file_path)
#     if df is not None:
#         prices[file_name] = df

In [ ]:
# results = {}
# for file, stock_price in prices.items():
#     print(f"Getting results for {file}")
#     testing_prices = stock_price.loc[stock_price.index < TESTING_CUTOFF]
#     testing_prices = testing_prices.iloc[:, testing_prices.columns.get_level_values(1) == "Adj Close"]
    
#     stocks = testing_prices.columns.get_level_values(0)
#     testing_prices.columns = stocks
#     pairs = pd.Series([(stocks[i], stocks[j]) for i in range(len(stocks)) for j in range(i+1, len(stocks))])
#     results[file] = find_suitable_pairs(testing_prices, pairs)

In [ ]:
# backtest_res = {}
# for file, (all_pairs_df, filtered_pairs_df) in results.items():
#     for pair in filtered_pairs_df['pair']:
#         print(f"Doing backtesting for: {pair}")
#         stock_prices = prices[file].loc[stock_price.index > TESTING_CUTOFF]
#         res = run_backtest(pair[0], pair[1], stock_prices[pair[0]], stock_prices[pair[1]], PARAMS)
#         backtest_res[file] = res
#         res.analyzers.sharpe.print()
#         res.analyzers.returns.print()
#         print('Ending test\n')
#         print('\n')
        